In [1]:
import numbers
import math

import numpy as np
import cv2 as cv
import scipy
import scipy.special
import scipy.spatial
import scipy.stats
import scipy.stats.qmc
import scipy.ndimage
import shapely
import shapely.geometry
import matplotlib
import matplotlib.patches
import matplotlib.patches as patches
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import matplotlib.collections as mc
from matplotlib.path import Path
import control
import control.matlab

import utility as util
import utility.npu

In [ ]:
"""
c.g. - center of gravity
x - position on x-axis in global frame
y - position on y-axis in global frame
psi - angle between car longitudinal axis and x-axis of global frame  
v - velocity at center of gravity
delta - angle between car front wheel and car longitudinal axis
beta - angle between car longitudinal axis and velocity vector from c.g.
L - length of vehicle
l_r - length between back wheel and c.g.
l_f - length between front wheel and c.g.
"""

sin = math.sin
cos = math.cos
atan = math.atan
tan = math.tan

def get_beta(delta, l_r=0.5, L=1.0):
    if l_r == L:
        return delta
    else:
        return atan((l_r / L)*tan(delta))

def get_dbeta_ddelta(delta, l_r=0.5, L=1.0):
    if l_r == L:
        return 1
    else:
        tan2 = tan(delta)**2
        return (1 + tan2) / ((L / l_r) + (l_r / L)*tan2)

def bicycle_kinematics(t, x, u, params):
    l_r = params.get('l_r', 0.5)
    L = params.get('L', 1)
    psi   = x[2]
    v     = x[3]
    delta = x[4]
    u_1, u_2 = u[0], u[1]
    beta  = get_beta(delta, l_r=l_r, L=L)
    return np.array([
        v*cos(psi + beta), # dot x 
        v*sin(psi + beta), # dot y
        (v / L)*cos(beta)*tan(delta), # dot psi
        u_1, # dot v
        u_2, # dot delta
    ])

def compute_nonlinear_dynamical_states(initial_state, T, Ts, U, l_r=0.5, L=1.0):
    timestamps = np.linspace(0, Ts*T, T + 1)
    mock_inputs = np.concatenate((U, np.array([0, 0])[None]), axis=0).T
    io_bicycle_kinematics = control.NonlinearIOSystem(
                bicycle_kinematics, None,
                inputs=('u_1', 'u_2'),
                outputs=('x', 'y', 'psi', 'v', 'delta'),
                states=('x', 'y', 'psi', 'v', 'delta'),
                params={'l_r': l_r, 'L': L},
                name='bicycle_kinematics')
    _, states = control.input_output_response(
            io_bicycle_kinematics, timestamps, mock_inputs, initial_state)
    return states.T

def get_state_matrix(x, y, psi, v, delta, l_r=0.5, L=1.0):
    beta   = get_beta(delta, l_r=l_r, L=L)
    dbeta  = get_dbeta_ddelta(delta, l_r=l_r, L=L)
    df3_dv = (1/L)*cos(beta)*tan(delta)
    df3_ddelta = (v/L)*(cos(beta)*(1 + tan(delta)**2) - sin(beta)*tan(delta)*dbeta)
    return np.array([
        # x, y, psi, v, delta
        [0, 0, -v*sin(psi + beta), cos(psi + beta), -v*sin(psi + beta)*dbeta],
        [0, 0,  v*cos(psi + beta), sin(psi + beta),  v*cos(psi + beta)*dbeta],
        [0, 0,                  0,          df3_dv,               df3_ddelta],
        [0, 0,                  0,               0,                        0],
    ])

def get_input_matrix():
    return np.array([
        [0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1],
    ]).T

def get_output_matrix():
    return np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
    ])

def get_feedforward_matrix():
    return np.array([
        [0, 0],
        [0, 0],
    ])

def cubic_interp_points(points):
    distance = np.cumsum( np.sqrt(np.sum( np.diff(points, axis=0)**2, axis=1 )) )
    distance = np.insert(distance, 0, 0)
    spline = scipy.interpolate.CubicSpline(distance, points, axis=0)
    return spline, distance

def draw_cars(ax, X, lw, color='blue'):
    vertex_set = util.npu.vertices_of_bboxes(X[:, :2], X[:, 2], lw)
    for idx, vertices in enumerate(vertex_set):
        patch = patches.Polygon(vertices, closed=True, color=color, fc='none')
        ax.add_patch(patch)

def disp_errors(ax, timesteps, X, X_gt, color='blue', label=None):
    disp_errors = np.linalg.norm(X_gt[:, :2] - X[:, :2], axis=1)
    ax.plot(timesteps, disp_errors, color=color, label=label)

def rot_errors(ax, timesteps, X, X_gt, color='blue', label=None):
    rot_errors  = np.abs(X_gt[:, 2] - X[:, 2])
    ax.plot(timesteps, rot_errors, color=color, label=label)